### Convolutional Neural Network

This is a basic convolutional approach to sequence prediction using convolutions through PyTorch!

In [41]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os, sys
import pickle

In [ ]:
# load data through the data preprocessor
sys.path.append(os.path.abspath('..'))  # add parent directory to sys.path
from data_cleanup import DataProcessor

processor = DataProcessor(input_steps=24, output_steps=1) # get 24 hour sequences, and a 1 hour output
Train, Val, Test = processor.load_and_process_data()

X_train, y_train = Train
X_val, y_val = Val
X_test, y_test = Test

print(f"Train: \n \t X: {X_train.shape}, y: {y_train.shape}")
print(f"Val: \n \t X: {X_val.shape}, y: {y_val.shape}")
print(f"Test: \n \t X: {X_test.shape}, y: {y_test.shape}")

Step 1/5: Fetching, cleaning, and engineering features...


/Users/felipejaracaceres/Documents/UCD/Machine Learning/ECS171G13/venv/lib/python3.11/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)
/Users/felipejaracaceres/Documents/UCD/Machine Learning/ECS171G13/data_cleanup.py:77: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


Step 2/5: Resampling data to hourly and setting 'Global_active_power' as target...


/Users/felipejaracaceres/Documents/UCD/Machine Learning/ECS171G13/data_cleanup.py:117: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.resample('H').agg(agg_dict)


Step 3/5: Splitting data and applying scaler...
Step 4/5: Creating time-series windows...
Step 5/5: Data processing complete.
Train: 
 	 X: (25903, 24, 8), y: (25903, 1)
Val: 
 	 X: (3600, 24, 8), y: (3600, 1)
Test: 
 	 X: (5014, 24, 8), y: (5014, 1)


/Users/felipejaracaceres/Documents/UCD/Machine Learning/ECS171G13/data_cleanup.py:118: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_hourly = df_hourly.fillna(method='ffill')


In [12]:
print(X_train[0].shape)
print(f"Sample tuple\n {X_train[0]}")

(24, 8)
Sample tuple
 [[0.63681623 0.27607416 0.33794529 0.63115734 0.         0.00681981
  0.49470253 0.47923051]
 [0.54504495 0.07832205 0.33550078 0.54148733 0.         0.14465183
  0.82477588 0.56052221]
 [0.50900588 0.08522501 0.28380167 0.50215208 0.         0.03086863
  0.81581092 0.58574856]
 [0.48854974 0.07177309 0.31598734 0.48110952 0.         0.
  0.8207009  0.57217659]
 [0.45559722 0.07385283 0.43441706 0.44990435 0.         0.00897344
  0.84189079 0.50306464]
 [0.32255458 0.04663923 0.49584732 0.32352941 0.         0.0028715
  0.21678892 0.48406388]
 [0.30103161 0.06690562 0.56718775 0.30105213 0.         0.00179469
  0.         0.51833529]
 [0.27320089 0.10805788 0.58031393 0.27343376 0.         0.01005025
  0.         0.46523825]
 [0.50110826 0.15385637 0.56061507 0.49294596 0.         0.54343144
  0.         0.46022976]
 [0.2273377  0.07593256 0.7666315  0.22835964 0.         0.01220388
  0.         0.38611807]
 [0.23897957 0.0776583  0.71656277 0.2404352  0.         

In [33]:
# Convolutional Neural Network: 
# The input will be a sequence of 24 hours of data, with 8 features/channels
# Slide a window through the hours and create data for the next layer 

class CNN(nn.Module): 
    def __init__(self):
        super(CNN, self).__init__() # Inherit from nn.Module
        # Define layers: 3 convolutions (1D) and 2 fully connected layers (very standard)
        self.conv1 = nn.Conv1d(in_channels=8, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=20, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2) # reusable pool layer
        self.fc1 = nn.Linear(in_features=(20), out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=1)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [36]:
# train
epochs = 10

net = CNN()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# turn data into torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
train_dataset = torch.utils.data.TensorDataset(X_train_t, y_train_t)

# use DataLoader, an iterable object from pytorch, that helps with batch training
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        # the inputs are given like (batch, time, features),
        # but the model expects (batch, features, time), permutate it: 
        inputs = inputs.permute(0, 2, 1)  

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.021
[1,  4000] loss: 0.020
[1,  6000] loss: 0.020
[2,  2000] loss: 0.019
[2,  4000] loss: 0.018
[2,  6000] loss: 0.017
[3,  2000] loss: 0.017
[3,  4000] loss: 0.016
[3,  6000] loss: 0.017
[4,  2000] loss: 0.016
[4,  4000] loss: 0.015
[4,  6000] loss: 0.015
[5,  2000] loss: 0.015
[5,  4000] loss: 0.014
[5,  6000] loss: 0.015
[6,  2000] loss: 0.015
[6,  4000] loss: 0.015
[6,  6000] loss: 0.014
[7,  2000] loss: 0.014
[7,  4000] loss: 0.014
[7,  6000] loss: 0.014
[8,  2000] loss: 0.014
[8,  4000] loss: 0.014
[8,  6000] loss: 0.014
[9,  2000] loss: 0.013
[9,  4000] loss: 0.014
[9,  6000] loss: 0.014
[10,  2000] loss: 0.013
[10,  4000] loss: 0.014
[10,  6000] loss: 0.013
Finished Training


In [39]:
# now validate: 
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)
test_dataset = torch.utils.data.TensorDataset(X_test_t, y_test_t)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

with torch.no_grad():
    test_loss = 0
    for data, label in test_loader:
        data = data.permute(0, 2, 1)
        y_pred = net(data)
        test_loss += F.mse_loss(y_pred, label)

print(f"Test Loss: {test_loss.item()}")


Test Loss: 1.336254596710205


hmmmmmm overfitting ...

#### Pickle 

Saving the model to pickle. This makes it easy to load the model later, if you guys want to test it out, or have your model being tested by somebody else! Pickle is a library that lets you store python objects in a file 

In [42]:
with open('net.pkl', 'wb') as f:
    pickle.dump(net, f)

In [43]:
# to load it in the notebook
with open('net.pkl', 'rb') as file:
    net = pickle.load(file)

# that's it now use 'net' as you would normally 